In [1]:
!gdown 18cij6foP2ilw2NtadINSnYP6UUbenXCf

Downloading...
From: https://drive.google.com/uc?id=18cij6foP2ilw2NtadINSnYP6UUbenXCf
To: /content/scc5966.zip
100% 18.4M/18.4M [00:00<00:00, 160MB/s]


In [2]:
!unzip scc5966.zip

Archive:  scc5966.zip
  inflating: movie_reviews.csv       
  inflating: movies_data.csv         
  inflating: test_data.csv           
  inflating: train_data .csv         
  inflating: users_data.csv          


In [3]:
import pandas as pd 

df_movie_reviews = pd.read_csv('movie_reviews.csv')

df_movies_data = pd.read_csv('movies_data.csv')

df_test_data = pd.read_csv('test_data.csv')

df_train_data = pd.read_csv('train_data .csv')

df_users_data = pd.read_csv('users_data.csv')

In [4]:
from tqdm import tqdm
df_user_item = pd.DataFrame(index = df_train_data['user_id'].unique(), columns=df_train_data['movie_id'].unique())

for movie in tqdm(df_train_data['movie_id'].unique()):
  temp = df_train_data[df_train_data.movie_id == movie]
  df_user_item.loc[temp.user_id, movie] = temp.rating.to_list()

100%|██████████| 3562/3562 [00:15<00:00, 224.25it/s]


In [ ]:
genres = set()
for item,row in df_movies_data.iterrows():
  genres_list = row.genres.split('|')
  for genre in genres_list:
    genres.add(genre)

In [ ]:
import pandas as pd
from tqdm import tqdm 

df=pd.DataFrame(index = df_train_data['movie_id'].unique(), columns=genres)

for movie in tqdm(df_train_data['movie_id'].unique()):
  temp = df_movies_data[df_movies_data.movie_id == movie].iloc[0]
  genres_list = temp.genres.split('|')
  for genre in genres_list:
    df.loc[movie, genre] = 1

100%|██████████| 3562/3562 [00:02<00:00, 1297.16it/s]


In [ ]:
df = df.fillna(0)

In [ ]:
from sklearn.metrics import jaccard_score
import numpy as np

In [ ]:
cont = 1
dic_sim = {}

for movie1 in tqdm(df.index):
  for movie2 in df.index[cont:]:
    x = np.array(df.loc[movie1].to_list())
    y = np.array(df.loc[movie2].to_list())
    
    similarity = jaccard_score(x,y)
    
    dic_sim[(movie1,movie2)] = similarity

  cont+=1

100%|██████████| 3562/3562 [1:39:28<00:00,  1.68s/it]


In [ ]:
import pickle

with open('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/similarities/total/dic_sim_items_genres_jaccard.p', 'wb') as fp:
    pickle.dump(dic_sim, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
import pickle

with open('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/similarities/total/dic_sim_items_genres_jaccard.p', 'rb') as fp:
    data = pickle.load(fp)

In [6]:
def similarity_itens(similarities, neighbor_item, item):
  
  if (item,neighbor_item) not in similarities.keys():
    similarity = similarities[(neighbor_item,item)]
  else:
    similarity = similarities[(item,neighbor_item)]
    
  return similarity
  
def k_neighbors_nearest(df, similarities, k, user, item):
  k_neighbors_similarity = [-2] * k
  k_neighbors = [-1] * k

  for neighbor_item in df.columns:
    
    ni = neighbor_item
    if item != ni and df.loc[user][ni] is not np.nan:
      
      similarity = similarity_itens(similarities, ni, item)

      for i in range(k):
        if similarity > k_neighbors_similarity[i]:
          aux = k_neighbors_similarity[i]
          k_neighbors_similarity[i] = similarity
          similarity = aux

          aux = k_neighbors[i]
          k_neighbors[i] = ni
          ni = aux

  return k_neighbors

def pred(df, similarities, k, user, item):

  k_neighbors = k_neighbors_nearest(df, similarities, k, user, item)
  
  sum = 0
  sum_similarity = 0

  for neighbor_item in k_neighbors:
    if(neighbor_item != -1): #se nao deu o numero maximo de vizinhos mais proximos
      rating_neighbor_item = df.loc[user][neighbor_item]
    
      similarity = similarity_itens(similarities, neighbor_item, item)

      sum_similarity+= similarity
      
      sum+= similarity * rating_neighbor_item
  
  if sum_similarity ==0 and sum ==0:
    return 0
  elif sum_similarity ==0 and sum !=0:
    return sum
  else:
    return sum/sum_similarity

In [9]:
import numpy as np

items_mean = np.mean(df_user_item)
user_mean = np.mean(df_user_item.T)
global_mean = np.nanmean(df_user_item.values.tolist())

In [13]:
def pred_generic(parameters):
  return pred(parameters[0], parameters[1], parameters[2], parameters[3], parameters[4])

In [14]:
def generate(parameters):
  l= []
  
  for item,row in tqdm(df_test_data.iterrows()):
    user = row['user_id']
    item = row['movie_id']
    if user not in df_train_data['user_id'].unique() and item in df_train_data['movie_id'].unique():
      l.append(items_mean[item])
    elif user in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
      l.append(user_mean[user])
    elif user not in df_train_data['user_id'].unique() and item not in df_train_data['movie_id'].unique():
      l.append(global_mean)
    else:
      parameters.append(user)
      parameters.append(item)

      value = pred_generic(parameters)
      if value < 1:
        value = 1
      if value > 5:
        value = 5

      l.append(value)
  
  return l

In [15]:
k = 12
  
y_pred = generate([df_user_item, data, k])
  
df_test_data['rating'] = y_pred
df_rating = df_test_data[['id','rating']]
df_rating = df_rating.set_index('id')
name = 'FBC_Genres_' + str(k) + '.csv'
df_rating.to_csv('/content/drive/MyDrive/USP/Doctorate/Credits/Recomender_Systems/results-kaggle/' +name)

3970it [24:46,  2.67it/s]
